In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

In [505]:
train = pd.read_csv("seasons-1314-1718.csv")
predict = pd.read_csv("season-1819_csv.csv")
#1-home win; 2-away win; 0-draw
train = train.replace({"FTR": {"H": 1, "A": 2, "D": 0}})
predict = predict.replace({"FTR": {"H": 1, "A": 2, "D": 0}})

In [476]:
train_labels = train['FTHG']

train_copy = train_copy.dropna(axis='columns')

data_dum = pd.get_dummies(train_copy, columns = train_copy.select_dtypes(include=['object']).columns.difference(['FTHG']))

data_dum = data_dum.drop('FTHG',axis=1)

X_train, X_test, y_train, y_test = train_test_split(data_dum,train_labels,test_size = 0.3, random_state = 0)

#model = KNeighborsClassifier(n_neighbors = 15)
#model = DecisionTreeClassifier(criterion="gini", max_depth=5, min_samples_leaf=500)
#model = BernoulliNB(alpha=11)
model = RandomForestClassifier(n_estimators=150, criterion="gini", max_depth=5, random_state=0)
model.fit(X_train, y_train)
#acc = accuracy_score(y_test, model.predict(X_test))
#print(acc)

0.3684210526315789


In [496]:
teams = train_copy.HomeTeam.unique()
zeroes = np.zeros((teams.size), dtype=int)
predicted_table=pd.DataFrame({"Team": teams, "Points": zeroes, "Goal difference": zeroes, "Goals for": zeroes, "Goals against": zeroes})

In [566]:
train_copy = train.copy()
predict_copy = predict.copy()
train_copy = train_copy.drop(['Div','Date','HTHG','HTAG','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR'],axis = 1)
predict_copy = predict_copy.drop(['Div','Date','HTHG','HTAG','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR'],axis = 1)
train_labels = train[["FTHG", "FTAG", "FTR"]]
train_copy = train_copy.dropna(axis='columns')
predict_copy = predict_copy.dropna(axis='columns')

data_dum_train = pd.get_dummies(train_copy, columns = train_copy.select_dtypes(include=['object']).columns.difference(["FTHG", "FTAG", "FTR"]))
data_dum_train = data_dum_train.drop("FTHG",axis=1)
data_dum_train = data_dum_train.drop("FTAG",axis=1)
data_dum_train = data_dum_train.drop("FTR",axis=1)
data_dum_predict = pd.get_dummies(predict_copy, columns = predict_copy.select_dtypes(include=['object']).columns.difference(["FTHG", "FTAG", "FTR"]))
data_dum_predict = data_dum_predict.drop("FTHG",axis=1)
data_dum_predict = data_dum_predict.drop("FTAG",axis=1)
data_dum_predict = data_dum_predict.drop("FTR",axis=1)

data_dum_train = data_dum_train.drop(columns=[col for col in data_dum_train if col not in list(data_dum_predict.columns)])
data_dum_predict = data_dum_predict.drop(columns=[col for col in data_dum_predict if col not in list(data_dum_train.columns)])

model = RandomForestClassifier(n_estimators=150, criterion="gini", max_depth=5, random_state=0)
model.fit(data_dum_train, train_labels)
prediction=model.predict(data_dum_predict)

for i in range(prediction.shape[0]):
    home = predict["HomeTeam"].iloc[i]
    away = predict["AwayTeam"].iloc[i]
    #add 1 point to both teams in case of draw
    if (prediction[i][2] == 0):
        predicted_table.loc[predicted_table["Team"] == home, ["Points"]] += 1
        predicted_table.loc[predicted_table["Team"] == away, ["Points"]] += 1
    #otherwise add 3 points to winner
    elif (prediction[i][2] == 1):
        predicted_table.loc[predicted_table["Team"] == home, ["Points"]] += 3
    else:
        predicted_table.loc[predicted_table["Team"] == away, ["Points"]] += 3
    #add goals for home to home team for goals and home team goal difference
    predicted_table.loc[predicted_table["Team"] == home, ["Goals for"]] += prediction[i][0]
    predicted_table.loc[predicted_table["Team"] == home, ["Goal difference"]] += prediction[i][0]
    #add goals for home to away team against goals and substract from away team goal difference
    predicted_table.loc[predicted_table["Team"] == away, ["Goals against"]] += prediction[i][0]
    predicted_table.loc[predicted_table["Team"] == away, ["Goal difference"]] -= prediction[i][0]
    #add goals for away to away team for goals and away team goal difference
    predicted_table.loc[predicted_table["Team"] == away, ["Goals for"]] += prediction[i][1]
    predicted_table.loc[predicted_table["Team"] == away, ["Goal difference"]] += prediction[i][1]
    #add goals for away to home team against goals and substract from home team goal difference
    predicted_table.loc[predicted_table["Team"] == home, ["Goals against"]] += prediction[i][1]
    predicted_table.loc[predicted_table["Team"] == home, ["Goal difference"]] -= prediction[i][1]

predicted_table

,Team,Points,Goal difference,Goals for,Goals against
0,Arsenal,128,25,50,25
1,Liverpool,149,58,69,11
2,Norwich,0,0,0,0
3,Sunderland,0,0,0,0
4,Swansea,0,0,0,0
5,West Brom,0,0,0,0
6,West Ham,86,-10,23,33
7,Chelsea,134,41,57,16
8,Crystal Palace,86,-10,24,34
9,Man City,149,49,52,3


In [570]:
predicted_table = predicted_table.sort_values(by=["Points"], ascending=False)
predicted_table = predicted_table.reset_index(drop=True)
predicted_table

,Team,Points,Goal difference,Goals for,Goals against
0,Man City,149,49,52,3
1,Liverpool,149,58,69,11
2,Chelsea,134,41,57,16
3,Tottenham,131,27,49,22
4,Arsenal,128,25,50,25
5,Man United,128,30,52,22
6,Leicester,98,-2,29,31
7,Everton,95,3,32,29
8,West Ham,86,-10,23,33
9,Bournemouth,86,-14,22,36
